# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

In [14]:
# load data from database
engine = create_engine('sqlite:///disasterinformer.db')
df = pd.read_sql("SELECT * FROM messages", engine)
X = df['message']
Y = df.drop(columns=['id', 'genre', 'message', 'original'])
titles = Y.columns
for title in titles:
    Y[title] = Y[title].astype('str').str.replace('2', '1')
    Y[title] = Y[title].astype('int')
titles

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

       
    clean_tokens = []
    for tok in tokens:
        if tok not in stopwords.words("english"):
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [19]:
for i in range(Y_pred.shape[1]):
    print('{}: ___________________________________'.format(titles[i]))
    report = classification_report(Y_pred[:,i], Y_test.values[:,i], target_names=titles)
    print(report)

related: ___________________________________
                        precision    recall  f1-score   support

               related       0.41      0.62      0.50      1014
               request       0.92      0.84      0.88      5540

           avg / total       0.84      0.80      0.82      6554

request: ___________________________________
                        precision    recall  f1-score   support

               related       0.98      0.89      0.94      6004
               request       0.42      0.83      0.56       550

           avg / total       0.94      0.89      0.90      6554

offer: ___________________________________
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6554
               request       0.00      0.00      0.00         0

           avg / total       1.00      1.00      1.00      6554

aid_related: ___________________________________
                        precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd1f400e510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [21]:
parameters = {
    'vect__ngram_range': [(1, 1)],
    'clf__estimator__min_samples_split': [2, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), total= 8.8min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 9.2min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 9.2min


[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed: 13.8min remaining: 13.8min


[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 9.2min
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), total= 4.4min
[CV]  clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), total= 4.2min


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed: 20.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__ngram_range': [(1, 1)], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [22]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.22983, std: 0.00305, params: {'clf__estimator__min_samples_split': 2, 'vect__ngram_range': (1, 1)},
 mean: 0.22302, std: 0.00430, params: {'clf__estimator__min_samples_split': 4, 'vect__ngram_range': (1, 1)}]

In [23]:
best_model = cv.best_estimator_
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
Y_pred = best_model.predict(X_test)

for i in range(Y_pred.shape[1]):
    print('{}: ___________________________________'.format(titles[i]))
    report = classification_report(Y_pred[:,i], Y_test.values[:,i], target_names=titles)
    print(report)

related: ___________________________________
                        precision    recall  f1-score   support

               related       0.41      0.63      0.49       993
               request       0.93      0.84      0.88      5561

           avg / total       0.85      0.80      0.82      6554

request: ___________________________________
                        precision    recall  f1-score   support

               related       0.97      0.89      0.93      5958
               request       0.41      0.77      0.54       596

           avg / total       0.92      0.88      0.90      6554

offer: ___________________________________
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6554
               request       0.00      0.00      0.00         0

           avg / total       1.00      1.00      1.00      6554

aid_related: ___________________________________
                        precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [25]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.